
#Ingest ciruits file using sparks dataframe reader

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date= dbutils.widgets.get("p_file_date")

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType
from pyspark.sql.functions import current_timestamp, lit

In [0]:
circuits_schema = StructType(fields=[StructField("circuitId", IntegerType(), False),
                                      StructField("circuitRef", StringType(), True),
                                      StructField("name", StringType(), True),
                                      StructField("location", StringType(), True),
                                      StructField("country", StringType(), True),
                                      StructField("lat", DoubleType(), True),
                                      StructField("lng", DoubleType(), True),
                                      StructField("alt", IntegerType(), True),
                                      StructField("url", StringType(), True)])

In [0]:
circuits_df = spark.read.csv(f"{raw_folder_path}/{v_file_date}/circuits.csv", header=True, schema=circuits_schema)

In [0]:
circuits_selected_df = circuits_df.select("circuitId", "circuitRef", "name", "location", "country", "lat", "lng", "alt")

In [0]:
circuits_renamed_df = circuits_df.withColumnRenamed("circuitId", "circuit_id") \
.withColumnRenamed("circuitRef", "circuit_ref") \
.withColumnRenamed("lat", "latitude") \
.withColumnRenamed("lng", "longitude") \
.withColumnRenamed("alt", "altitude") \
.withColumn("data_source", lit(v_data_source)) \
.withColumn("file_date", lit(v_file_date))    


# add ingestion column


In [0]:
circuits_final_df = add_ingestion_date(circuits_renamed_df)



# write to datalake as delta file


In [0]:
# Normal Mode

#circuits_final_df.write.parquet(f"{processed_folder_path}/circuits", mode="overwrite")

In [0]:
# Using SparkSQL method

circuits_final_df.write.mode("overwrite")\
                       .format("delta")\
                       .saveAsTable("f1_processed.circuits")

In [0]:
# display(spark.read.parquet(f"{processed_folder_path}/circuits"))

In [0]:
%sql
SELECT * FROM f1_processed.circuits;

In [0]:
dbutils.notebook.exit("Success")